SETTING UP THE ENVIRONMENT

In [5]:
!pip install langchain
!pip install cohere tiktoken openai

In [6]:
!pip install python-dotenv

In [7]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

In [8]:
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [9]:
import pandas as pd


In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
#MOUNTING OUR DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
df = pd.read_csv("/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv", encoding='latin-1')

In [13]:
df.shape


(12625, 13)

In [14]:
df.head()

,comment,comment_id,like_count,reply_count,published_at,parent_id,video_id,title,channel,publish_date,views,likes,comments_count
0,"So , do we have a forest in Ghana by name WONTUMI FOREST ? WOW, GHANAIANS ARE NOT SERIOUS AT ALL COST",UgwwfkJtgmu2XWedKG54AaABAg,0.0,0.0,2024-10-17T19:17:10Z,NaN,m3Q-UZQF56M,Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24),JoyNews,10/11/2024 19:35,10422.0,65.0,28.0
1,Backwards society of leaders... polluting your own water bodies and calling for God's intervention. This is a man made disaster and man must take it's own action. It has nothing to do with God.????,UgwU_kn1hnQ020jABdx4AaABAg,2.0,1.0,2024-10-15T19:08:41Z,NaN,m3Q-UZQF56M,Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24),JoyNews,10/11/2024 19:35,10422.0,65.0,28.0
2,Exactly ??,UgwU_kn1hnQ020jABdx4AaABAg.A9cUk3_TA8xA9jXMYU8VHm,0.0,0.0,2024-10-18T12:46:14Z,UgwU_kn1hnQ020jABdx4AaABAg,m3Q-UZQF56M,Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24),JoyNews,10/11/2024 19:35,10422.0,65.0,28.0
3,kpaako???,UgzDOCWqsoEA5kaCZpN4AaABAg,0.0,0.0,2024-10-13T04:05:41Z,NaN,m3Q-UZQF56M,Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24),JoyNews,10/11/2024 19:35,10422.0,65.0,28.0
4,YOU CAN'T DO THE RIVERS AND LEAVE THE FOREST UNTIL ANOTHER TIME. IT WILL BE BETTER IF YOU DON'T START AT ALL. THE PRESSURE THAT WILL COME ON THE FOREST EVEN IF ONE WEEK IS ALLOWED WILL BE VERY DISASTROUS. UNLESS YOU WANT GHANAIANS TO KNOW THAT SOME ARE BEING PROTECTED.,Ugw-i4E2ucWcPSsKV6J4AaABAg,1.0,0.0,2024-10-12T16:35:54Z,NaN,m3Q-UZQF56M,Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24),JoyNews,10/11/2024 19:35,10422.0,65.0,28.0


LANG CHAIN

In [15]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [16]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', encoding='latin-1')

In [17]:
docs=loader.load()

In [18]:
len(docs)

12625

In [19]:
print(docs[0].page_content[:1000])

comment: So , do we have a forest in Ghana  by name WONTUMI FOREST ?  WOW,  GHANAIANS ARE NOT SERIOUS  AT ALL  COST
comment_id: UgwwfkJtgmu2XWedKG54AaABAg
like_count: 0
reply_count: 0
published_at: 2024-10-17T19:17:10Z
parent_id: 
video_id: m3Q-UZQF56M
title: Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24)
channel: JoyNews
publish_date: 10/11/2024 19:35
views: 10422
likes: 65
comments_count: 28


SPLITTING DOCS INTO CHUCKS BEFORE EMBEDDING

In [20]:
chunk_size = 256
chunk_overlap = 32

In [21]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [22]:
c_text_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator=' ',
    length_function=len
    )

In [23]:
r_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    add_start_index=True

)


In [24]:
pages = c_text_splitter.split_documents(docs)

In [25]:
print(docs[0])

page_content='comment: So , do we have a forest in Ghana  by name WONTUMI FOREST ?  WOW,  GHANAIANS ARE NOT SERIOUS  AT ALL  COST
comment_id: UgwwfkJtgmu2XWedKG54AaABAg
like_count: 0
reply_count: 0
published_at: 2024-10-17T19:17:10Z
parent_id: 
video_id: m3Q-UZQF56M
title: Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24)
channel: JoyNews
publish_date: 10/11/2024 19:35
views: 10422
likes: 65
comments_count: 28' metadata={'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'row': 0}


In [26]:
print(pages[0])

page_content='comment: So , do we have a forest in Ghana by name WONTUMI FOREST ? WOW, GHANAIANS ARE NOT SERIOUS AT ALL COST
comment_id: UgwwfkJtgmu2XWedKG54AaABAg
like_count: 0
reply_count: 0
published_at: 2024-10-17T19:17:10Z
parent_id: 
video_id: m3Q-UZQF56M
title:' metadata={'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'row': 0}


In [27]:
print(pages[1])

page_content='video_id: m3Q-UZQF56M
title: Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24)
channel: JoyNews
publish_date: 10/11/2024 19:35
views: 10422
likes: 65
comments_count: 28' metadata={'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'row': 0}


In [28]:
pages = r_text_splitter.split_documents(docs)

In [29]:
print(pages[0])

page_content='comment: So , do we have a forest in Ghana  by name WONTUMI FOREST ?  WOW,  GHANAIANS ARE NOT SERIOUS  AT ALL  COST
comment_id: UgwwfkJtgmu2XWedKG54AaABAg
like_count: 0
reply_count: 0
published_at: 2024-10-17T19:17:10Z
parent_id: 
video_id: m3Q-UZQF56M' metadata={'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'row': 0, 'start_index': 0}


In [30]:
print(pages[1])

page_content='video_id: m3Q-UZQF56M
title: Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24)
channel: JoyNews
publish_date: 10/11/2024 19:35
views: 10422
likes: 65
comments_count: 28' metadata={'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'row': 0, 'start_index': 231}


In [31]:
print(len(docs))
print(len(pages))

12625
18569


TURNING WORDS INTO NUMBERS - EMBEDDINGS

In [32]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:0

In [33]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [34]:
embeddings = OpenAIEmbeddings()

<ipython-input-34-73ad2f8e367a>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


FOLDER TO STORE EMBEDDINGS

In [35]:
persist_directory = 'persist_chroma'

CONVERT EMBEDDINGS INTO DB and store in directory for future use

In [36]:
vectordb = Chroma.from_documents(
    documents=pages,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [38]:
print(vectordb._collection.count())

18569


In [39]:
vectordb.persist()

<ipython-input-39-d832e715b29a>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


Querying our vector database

In [40]:
question="which comments talk about water bodies?"

In [41]:
docs = vectordb.similarity_search(question, k=3) #k top number to return

In [42]:
docs

[Document(metadata={'row': 9599, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: Only major water bodies be guarded? Smaller water bodies also necessary because the smaller water joins the bigger one n all join the mighty sea\ncomment_id: Ugwcf_KJnatp6AJmyC54AaABAg\nlike_count: 0\nreply_count: 0\npublished_at: 2021-05-04T22:34:10Z'),
 Document(metadata={'row': 11801, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: You won\x92t believe how the water bodies are being destroyed\ncomment_id: Ugz_kVsewLFk_V12W314AaABAg\nlike_count: 6\nreply_count: 0\npublished_at: 2024-09-04T18:44:18Z\nparent_id: \nvideo_id: nCR9LrWz79M'),
 Document(metadata={'row': 9292, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: I always cry when i see water bodies like this????\ncomment_id: Ugy-Qa8ESkzKrElMJj94AaABAg\nlike_count: 

SIMILARITY CHECK

In [43]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings

)

<ipython-input-43-c2a71514b216>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [44]:
question = "Comments that talk about governance?"

In [45]:
doc_similarity = vectordb.similarity_search_with_score(question,k=3)
doc_similarity

[(Document(metadata={'row': 12357, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content="comment: Government controlled everything and shared among themselves the leaders don't think about the country they worry about their fat bank account . Shame !\ncomment_id: Ugw9bCMhfDor152zth54AaABAg\nlike_count: 0\nreply_count: 0"),
  0.335551381111145),
 (Document(metadata={'row': 9803, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: @@abdulmumuni1750 did he mention government, he said leaders are the chiefs not leaders too ?\ncomment_id: UgzJlSVUPjMo9Tqdu0F4AaABAg.9Mnu57hyv6H9Xa6yw9nSkU\nlike_count: 0\nreply_count: 0\npublished_at: 2022-01-24T16:06:44Z'),
  0.34302911162376404),
 (Document(metadata={'row': 11222, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: Everything uncivilized citizens do we turn to blame leaders... 

MAX MARGINAL RELEVANCE - MMR

In [46]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)
docs_mmr

[Document(metadata={'row': 12357, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content="comment: Government controlled everything and shared among themselves the leaders don't think about the country they worry about their fat bank account . Shame !\ncomment_id: Ugw9bCMhfDor152zth54AaABAg\nlike_count: 0\nreply_count: 0"),
 Document(metadata={'row': 10816, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: Prayer is useless in of governance\ncomment_id: UgwGGudw17IRmosSD2R4AaABAg\nlike_count: 0\nreply_count: 0\npublished_at: 2022-10-17T19:06:20Z\nparent_id: \nvideo_id: RmYyIX8coXs'),
 Document(metadata={'row': 11747, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: When you live with selfish nd careless governors this is what happens so I agree with you Maghraheb.??????\ncomment_id: Ugw4fNCeRGcevG5HcJF4AaABAg\nlike_coun